In [1]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 
from gensim.corpora.mmcorpus import MmCorpus
import helper
import corpora


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension(s)as a python packages.
  "as a python packages.", UserWarning)


In [4]:
#%unload_ext autotime

time: 827 µs


In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

time: 76.7 ms


In [6]:
pwd

u'/home/jshoun01/Data/metis/ds4_fletcher/analysis'

time: 92.1 ms


In [7]:
ls ../data

Badges.xml*    makedb.py*        Posts.xml*      stopwords.txt  Votes.xml*
Comments.xml*  PostHistory.xml*  so-dump.db*     Tags.xml*
data.sqlite    PostLinks.xml*    so-parser.log*  Users.xml*
time: 217 ms


In [8]:
data_dir='data'

time: 2.26 ms


## Normalizer

In [9]:
options = {
    "stem":False
    ,"stop_words":"../data/stopwords.txt"
    ,"strip_html_tags":True
    ,"filter_parts_of_speech":True
    ,"min_word_length":1
    ,"lower_case":True
    
}

time: 97.6 ms


In [10]:
ag_preprocesser = helper.Normalizer(**options)

time: 92.7 ms


In [11]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)

[u'vista',
 u'virtual',
 u'machine',
 u'use',
 u'gb',
 u'virtual',
 u'hard',
 u'disk',
 u'worried',
 u'running',
 u'space',
 u'time',
 u'used',
 u'treesize',
 u'check',
 u'culprits..',
 u'primary',
 u'culprit',
 u'/winsxs',
 u'windows',
 u'side-by-side',
 u'folder',
 u'windows',
 u'side',
 u'side',
 u'folder',
 u'well',
 u'explained',
 u'pretty',
 u'well',
 u'components',
 u'operating',
 u'system',
 u'found',
 u'winsxs',
 u'folder',
 u'\\u2013',
 u'fact',
 u'call',
 u'location',
 u'component',
 u'store',
 u'component',
 u'unique',
 u'name',
 u'includes',
 u'version',
 u'language',
 u'processor',
 u'architecture',
 u'built',
 u'winsxs',
 u'folder',
 u'location',
 u'component',
 u'found',
 u'system',
 u'instances',
 u'files',
 u'see',
 u'system',
 u'\\u201cprojected\\u201d',
 u'hard',
 u'linking',
 u'component',
 u'store',
 u'let',
 u'repeat',
 u'last',
 u'point',
 u'\\u2013',
 u'one',
 u'instance',
 u'full',
 u'data',
 u'copy',
 u'version',
 u'file',
 u'os',
 u'instance',
 u'located',
 

time: 163 ms


## Loading Streaming

In [12]:
db_path ='../data/data.sqlite'

time: 44.3 ms


In [13]:
posts_corpus= corpora.SuperUserSqliteCorpus(
    db_path=db_path
    ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()


Iterated over 1000 of 999 docs
Iterated over 2000 of 1999 docs
Iterated over 3000 of 2999 docs
Iterated over 4000 of 3999 docs
Iterated over 5000 of 4999 docs
Iterated over 6000 of 5999 docs
Iterated over 7000 of 6999 docs
Iterated over 8000 of 7999 docs
Iterated over 9000 of 8999 docs
Iterated over 10000 of 9999 docs
Iterated over 11000 of 10999 docs
Iterated over 12000 of 11999 docs
Iterated over 13000 of 12999 docs
Iterated over 14000 of 13999 docs
Iterated over 15000 of 14999 docs
Iterated over 16000 of 15999 docs
Iterated over 17000 of 16999 docs
Iterated over 18000 of 17999 docs
Iterated over 19000 of 18999 docs
Iterated over 20000 of 19999 docs
Iterated over 21000 of 20999 docs
Iterated over 22000 of 21999 docs
Iterated over 23000 of 22999 docs
Iterated over 24000 of 23999 docs
Iterated over 25000 of 24999 docs
Iterated over 26000 of 25999 docs
Iterated over 27000 of 26999 docs
Iterated over 28000 of 27999 docs
Iterated over 29000 of 28999 docs
Iterated over 30000 of 29999 docs


In [14]:
"""
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                ,clip_docs=100000
                                #,clip_docs=1000
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()
"""

"\npost_file=os.path.join(data_dir,'Posts.xml')\nposts_corpus = helper.XmlCorpus(post_file\n                                ,clip_docs=100000\n                                #,clip_docs=1000\n                                ,body_field='Body'\n                                ,normalizer=ag_preprocesser)\nposts_corpus.finalize_dictionary()\n"

time: 1.85 ms


In [15]:
"""
num_documents_processed=0
counter=0
for vector in posts_corpus: # load one vector into memory at a time
    print ("Item %s"%(counter))
    print(vector)
    print()
    counter=counter+1
"""

'\nnum_documents_processed=0\ncounter=0\nfor vector in posts_corpus: # load one vector into memory at a time\n    print ("Item %s"%(counter))\n    print(vector)\n    print()\n    counter=counter+1\n'

time: 103 ms


In [16]:
print ("Number of docs in collection: %s"%(len(posts_corpus)))

Number of docs in collection: 257118
time: 93.8 ms


In [17]:
?gensim.models.LdaModel

time: 94.5 ms


In [18]:
?gensim.models.LdaMulticore

time: 105 ms


In [19]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 10

#lda = gensim.models.LdaModel(posts_corpus, num_topics=5, 
#                            id2word=posts_corpus.dictionary
#                                ,passes=2
#                                )



time: 80.4 ms


In [20]:
lda = gensim.models.LdaMulticore(posts_corpus, num_topics=num_topics, 
                            id2word=posts_corpus.dictionary
                                 ,passes=10
                                )

Iterated over 1000 of 257118 docs
Iterated over 2000 of 257118 docs
Iterated over 3000 of 257118 docs
Iterated over 4000 of 257118 docs
Iterated over 5000 of 257118 docs
Iterated over 6000 of 257118 docs
Iterated over 7000 of 257118 docs
Iterated over 8000 of 257118 docs
Iterated over 9000 of 257118 docs
Iterated over 10000 of 257118 docs
Iterated over 11000 of 257118 docs
Iterated over 12000 of 257118 docs
Iterated over 13000 of 257118 docs
Iterated over 14000 of 257118 docs
Iterated over 15000 of 257118 docs
Iterated over 16000 of 257118 docs
Iterated over 17000 of 257118 docs
Iterated over 18000 of 257118 docs
Iterated over 19000 of 257118 docs
Iterated over 20000 of 257118 docs
Iterated over 21000 of 257118 docs
Iterated over 22000 of 257118 docs
Iterated over 23000 of 257118 docs
Iterated over 24000 of 257118 docs
Iterated over 25000 of 257118 docs
Iterated over 26000 of 257118 docs
Iterated over 27000 of 257118 docs
Iterated over 28000 of 257118 docs
Iterated over 29000 of 257118

In [21]:
?lda.show_topics

time: 2.27 ms


In [22]:
lda.show_topics(num_topics)

[u'0.035*windows + 0.029*drive + 0.017*disk + 0.016*boot + 0.013*partition + 0.012*hard + 0.011*system + 0.010*install + 0.010*usb + 0.008*ubuntu',
 u'0.026*network + 0.019*router + 0.019*ip + 0.017*connection + 0.015*connect + 0.014*internet + 0.012*port + 0.012*address + 0.011*server + 0.010*wireless',
 u'0.016*kernel + 0.014*bytes + 0.012*device + 0.011*id + 0.007*ff + 0.007*error + 0.006*info + 0.006*size + 0.006*name + 0.006*yes',
 u'0.023*command + 0.018*file + 0.014*error + 0.010*root + 0.010*script + 0.010*line + 0.009*run + 0.008*directory + 0.007*get + 0.007*using',
 u'0.030*video + 0.019*audio + 0.018*stream + 0.014*ffmpeg + 0.011*player + 0.010*media + 0.009*play + 0.008*output + 0.008*debug + 0.007*videos',
 u'0.048*file + 0.041*files + 0.021*folder + 0.012*like + 0.011*would + 0.010*want + 0.010*way + 0.008*one + 0.007*using + 0.007*directory',
 u'0.013*excel + 0.010*text + 0.010*data + 0.010*word + 0.010*like + 0.009*want + 0.009*outlook + 0.008*email + 0.008*column + 0.

time: 126 ms


In [23]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['windows', 'drive', 'disk', 'boot', 'partition', 'hard', 'system', 'install', 'usb', 'ubuntu', 'linux', 'os', 'installed', 'data', 'hdd', 'using', 'ssd', 'tried', 'new', 'drives']
()
['network', 'router', 'ip', 'connection', 'connect', 'internet', 'port', 'address', 'server', 'wireless', 'using', 'connected', 'access', 'wifi', 'use', 'dns', 'host', 'vpn', 'ping', 'one']
()
['kernel', 'bytes', 'device', 'id', 'ff', 'error', 'info', 'size', 'name', 'yes', 'root', 'link', 'type', 'memory', 'address', 'version', 'jan', 'enabled', 'state', 'kb']
()
['command', 'file', 'error', 'root', 'script', 'line', 'run', 'directory', 'get', 'using', 'set', 'install', 'following', 'sudo', 'echo', 'debug1', 'version', 'path', 'linux', 'ssh']
()
['video', 'audio', 'stream', 'ffmpeg', 'player', 'media', 'play', 'output', 'debug', 'videos', 'http', 'input', 'vlc', 'using', 'file', '-i', 'main', 'libx264', 'metadata', 'kb/s']
()
['file', 'files', 'folder', 'like', 'would', 'want', 'way', 'one', 'using', 'di

In [24]:
posts_corpus_dict = posts_corpus.dictionary

time: 78.1 ms


In [25]:
posts_corpus_dict.save('posts_corpus.dict')

time: 215 ms


In [26]:
lda.show_topics()

[u'0.035*windows + 0.029*drive + 0.017*disk + 0.016*boot + 0.013*partition + 0.012*hard + 0.011*system + 0.010*install + 0.010*usb + 0.008*ubuntu',
 u'0.026*network + 0.019*router + 0.019*ip + 0.017*connection + 0.015*connect + 0.014*internet + 0.012*port + 0.012*address + 0.011*server + 0.010*wireless',
 u'0.016*kernel + 0.014*bytes + 0.012*device + 0.011*id + 0.007*ff + 0.007*error + 0.006*info + 0.006*size + 0.006*name + 0.006*yes',
 u'0.023*command + 0.018*file + 0.014*error + 0.010*root + 0.010*script + 0.010*line + 0.009*run + 0.008*directory + 0.007*get + 0.007*using',
 u'0.030*video + 0.019*audio + 0.018*stream + 0.014*ffmpeg + 0.011*player + 0.010*media + 0.009*play + 0.008*output + 0.008*debug + 0.007*videos',
 u'0.048*file + 0.041*files + 0.021*folder + 0.012*like + 0.011*would + 0.010*want + 0.010*way + 0.008*one + 0.007*using + 0.007*directory',
 u'0.013*excel + 0.010*text + 0.010*data + 0.010*word + 0.010*like + 0.009*want + 0.009*outlook + 0.008*email + 0.008*column + 0.

time: 70.4 ms


In [31]:
model_serialized_path='lda_topic_%s.lda'%(num_topics)
lda.save(model_serialized_path) 

time: 114 ms


In [32]:
lda = gensim.models.LdaMulticore.load(model_serialized_path)

time: 91.8 ms


In [33]:
lda.show_topics()

[u'0.035*windows + 0.029*drive + 0.017*disk + 0.016*boot + 0.013*partition + 0.012*hard + 0.011*system + 0.010*install + 0.010*usb + 0.008*ubuntu',
 u'0.026*network + 0.019*router + 0.019*ip + 0.017*connection + 0.015*connect + 0.014*internet + 0.012*port + 0.012*address + 0.011*server + 0.010*wireless',
 u'0.016*kernel + 0.014*bytes + 0.012*device + 0.011*id + 0.007*ff + 0.007*error + 0.006*info + 0.006*size + 0.006*name + 0.006*yes',
 u'0.023*command + 0.018*file + 0.014*error + 0.010*root + 0.010*script + 0.010*line + 0.009*run + 0.008*directory + 0.007*get + 0.007*using',
 u'0.030*video + 0.019*audio + 0.018*stream + 0.014*ffmpeg + 0.011*player + 0.010*media + 0.009*play + 0.008*output + 0.008*debug + 0.007*videos',
 u'0.048*file + 0.041*files + 0.021*folder + 0.012*like + 0.011*would + 0.010*want + 0.010*way + 0.008*one + 0.007*using + 0.007*directory',
 u'0.013*excel + 0.010*text + 0.010*data + 0.010*word + 0.010*like + 0.009*want + 0.009*outlook + 0.008*email + 0.008*column + 0.

time: 27.9 ms


In [34]:
sample_post="""

Can't insert cells in Excel 2010 - “operation not allowed” error message

I was working on a spreadsheet in Excel 2010, and all of a sudden when I attempted to insert a new row of cells, I saw that the insert and delete options were grayed out.

I attempted to copy a different row and insert it as a new row, but I got the error message:

"This operation is not allowed. The operation is attempting to shift cells in a table on your worksheet."

I have not merged or hidden any cells/rows/columns. There are no formulas. There is no data verification. I tried closing and re-opening the spreadsheet.

Searching for answers brings up nothing useful.
As the message indicates, you're trying to insert rows into a Table in your sheet. Typically, but not necessarily, tables will have banded formatting. When you click in a cell in a table, the Table tab will appear, like so:


"""




time: 1.83 ms


In [35]:
sample_post="""
I start sshd on 192.168.1.8 with the following:

sudo /usr/sbin/sshd -ddd
I then try to open an SSH tunnel to 192.168.1.5:1234 and am unable to do so. Here's what sshd -ddd says:

debug1: server_input_channel_open: confirm direct-tcpip
debug3: channel 1: waiting for connection
debug1: channel 1: connected to 192.168.1.5 port 1234
debug2: channel 1: read<=0 rd 10 len 0
debug2: channel 1: read failed
debug2: channel 1: close_read
debug2: channel 1: input open -> drain
debug2: channel 1: ibuf empty
debug2: channel 1: send eof
debug2: channel 1: input drain -> closed
debug2: channel 1: rcvd close
debug2: channel 1: output open -> drain
debug3: channel 1: will not send data after close
debug2: channel 1: obuf empty
debug2: channel 1: close_write
debug2: channel 1: output drain -> closed
debug2: channel 1: send close
debug2: channel 1: is dead
debug2: channel 1: garbage collecting
debug1: channel 1: free: direct-tcpip, nchannels 2
debug3: channel 1: status: The following connections are open:
  #0 server-session (t4 r256 i0/0 o0/0 fd 11/9 cc -1)
  #1 direct-tcpip (t4 r257 i3/0 o3/0 fd 10/10 cc -1)
So it's able to connect but the read is failing. And yet I'm able to connect (with no failing reads) when I do this (on 192.168.1.8):

ssh -p 1234 username@192.168.1.5
So why is the tunnel failing on 192.168.1.8 when the direct ssh connection on 192.168.1.8 isn't?

I'm using PuTTY to create the tunnel.



what is the exact command you type when you "open an SSH tunnel"? –  Jakuje 34 mins ago
  	 	
I'm using PuTTY to dynamically create the tunnel on-demand. PuTTY creates a SOCKS5 server that I send requests to and read responses from. I then use a PHP script I wrote to connect to the SOCKS5 server, which in turn, tells PuTTY to request the tunnel. And PuTTY is creating the tunnel, as demonstrated above. I can successfully create tunnel's to google.com:80 and am able to send an HTTP request and read the response but not so for 192.168.1.5:1234. At minimum I should be getting the SSH server identifier (SSH-2.0-whatever) but I'm not. –  neubert 26 mins ago

"""




time: 4.51 ms


In [36]:
processed_tokens=ag_preprocesser.process(sample_post)
doc_bow=posts_corpus_dict.doc2bow(processed_tokens)

time: 9.58 ms


In [37]:
import operator 

time: 991 µs


In [38]:
lda[doc_bow]

[(1, 0.26827428256353969), (2, 0.080848446693278619), (3, 0.64717255846718713)]

time: 4.01 ms


In [39]:
sorted(lda[doc_bow],key=operator.itemgetter(1),reverse=True)

[(3, 0.64716523153501404), (1, 0.2682707875161815), (2, 0.080859268710276197)]

time: 4.13 ms


In [ ]:
?lda.top_topics

In [40]:
lda.show_topic(3)

[(0.023383455820639977, u'command'),
 (0.018098584877683819, u'file'),
 (0.014268151178736106, u'error'),
 (0.010456277336653531, u'root'),
 (0.010362130543246641, u'script'),
 (0.010152097189539827, u'line'),
 (0.0087336786749196704, u'run'),
 (0.0077456837849114255, u'directory'),
 (0.0072975574387009684, u'get'),
 (0.0072519370472943576, u'using')]

time: 4.59 ms


In [ ]:
lda.show_topic(0)

In [ ]:
lda.num_topics

In [42]:
!date

Mon Oct 12 18:08:33 EDT 2015
time: 128 ms


In [43]:
top_topics=lda.top_topics(posts_corpus)

Iterated over 1000 of 257118 docs
Iterated over 2000 of 257118 docs
Iterated over 3000 of 257118 docs
Iterated over 4000 of 257118 docs
Iterated over 5000 of 257118 docs
Iterated over 6000 of 257118 docs
Iterated over 7000 of 257118 docs
Iterated over 8000 of 257118 docs
Iterated over 9000 of 257118 docs
Iterated over 10000 of 257118 docs
Iterated over 11000 of 257118 docs
Iterated over 12000 of 257118 docs
Iterated over 13000 of 257118 docs
Iterated over 14000 of 257118 docs
Iterated over 15000 of 257118 docs
Iterated over 16000 of 257118 docs
Iterated over 17000 of 257118 docs
Iterated over 18000 of 257118 docs
Iterated over 19000 of 257118 docs
Iterated over 20000 of 257118 docs
Iterated over 21000 of 257118 docs
Iterated over 22000 of 257118 docs
Iterated over 23000 of 257118 docs
Iterated over 24000 of 257118 docs
Iterated over 25000 of 257118 docs
Iterated over 26000 of 257118 docs
Iterated over 27000 of 257118 docs
Iterated over 28000 of 257118 docs
Iterated over 29000 of 257118

In [44]:
!date

Wed Oct 14 00:17:05 EDT 2015
time: 135 ms


In [45]:
top_topics

[([(0.026360114487007473, u'network'),
   (0.018964691228183131, u'router'),
   (0.018668700784420178, u'ip'),
   (0.017309162972512608, u'connection'),
   (0.014726757160526651, u'connect'),
   (0.013954085575785896, u'internet'),
   (0.012410631658168702, u'port'),
   (0.011603014244405886, u'address'),
   (0.011447982552955142, u'server'),
   (0.0097860805377447246, u'wireless'),
   (0.0085482941347489374, u'using'),
   (0.0085069619094518302, u'connected'),
   (0.0081539253391243755, u'access'),
   (0.0077688263510113749, u'wifi'),
   (0.006979315326592319, u'use'),
   (0.0065813931290753689, u'dns'),
   (0.0064989342568647895, u'host'),
   (0.006340129385896248, u'vpn'),
   (0.0056992002922746123, u'ping'),
   (0.0056738449108135408, u'one')],
  -659.68899319368177),
 ([(0.020613231070223486, u'windows'),
   (0.02026976779799974, u'server'),
   (0.013900033021548347, u'user'),
   (0.010009198998610198, u'access'),
   (0.0079469101139447561, u'password'),
   (0.0077774440197467942,

time: 22 ms


In [46]:
?lda.top_topics

time: 2.22 ms


In [47]:
len(top_topics)

10

time: 1.74 ms
